In [1]:
#ADD SCHEDULER

import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import yfinance as yf
import pandas_datareader as web
from pandas_datareader import data
from bs4 import BeautifulSoup as bs
from scipy import stats
tick = pd.read_csv("all_stocks")
final = pd.DataFrame(columns=[
                            'Ticker',
                            'Price',
                            '1y Price Return',
                            '1y percentile', 
                            '6m Price Return', 
                            '6m percentile', 
                            '3m Price Return', 
                            '3m percentile',
                            '1m Price Return', 
                            '1m percentile'
                            ])
for i in range(len(tick["0"])):
    print("Producing data for: ", tick["0"][i])
    try:
        max = yf.Ticker(tick["0"][i]).history(period='max')
        d = yf.Ticker(tick["0"][i]).history(period='max')
        y = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
        m6 = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
        m3 = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=3))
        m1 = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=1))
    except IndexError: 
        print("Couldn't find: ",tick["0"][i])
        continue
    except KeyError:
        continue
    
    max.reset_index(inplace = True)
    d.reset_index(inplace=True)
    y.reset_index(inplace=True)
    m6.reset_index(inplace=True)
    m3.reset_index(inplace=True)
    m1.reset_index(inplace=True)

    d['Ticker'] = tick['0'][i]    
    try: 
        perc_change1y = ((y.iloc[-1]['Close'] - y.iloc[0]['Close'])/y.iloc[0]["Close"])
        d['1y Price Return'] = perc_change1y
        perc_change6m = ((m6.iloc[-1]['Close'] - m6.iloc[0]['Close'])/m6.iloc[0]["Close"])
        d['6m Price Return'] = perc_change6m
        perc_change3m = ((m3.iloc[-1]['Close'] - m3.iloc[0]['Close'])/m3.iloc[0]["Close"])
        d['3m Price Return'] = perc_change3m
        perc_change1m = ((m1.iloc[-1]['Close'] - m1.iloc[0]['Close'])/m1.iloc[0]["Close"])
        d['1m Price Return'] = perc_change1m

        d['Price'] = d['Close'] 
        d['1y percentile'] = 0
        d['6m percentile'] = 0
        d['3m percentile'] = 0
        d['1m percentile'] = 0
        d = d.iloc[-1]
    except IndexError: 
        continue
    #print(d)

    #Change to keep only last 2 days!!! get date
    
    #Append latest value
    final = final.append(
        pd.Series(d,
        index=['Ticker',
                            'Price',
                            '1y Price Return',
                            '1y percentile', 
                            '6m Price Return', 
                            '6m percentile', 
                            '3m Price Return', 
                            '3m percentile',
                            '1m Price Return', 
                            '1m percentile']), 
        ignore_index = True
    ) 
    print(y)
    break 
final.to_csv("momentum_advanced_sheet")

#Get percentiles for each stock
cols = ['1y', '6m', '3m', '1m']
top90 = []
for c in cols: 
    for index, row in final.iterrows(): 
        #print(row['Ticker'])
        percentile_change = stats.percentileofscore(final[c+ ' Price Return'], final[c + ' Price Return'].loc[index])
        final[c + ' percentile'][index] = percentile_change 
final['Shares To Buy'] = 0

#Calculate HQM score
#Get the mean of all 4 percentiles 
final['HQM score'] = 0
from statistics import mean
for index, row in final.iterrows(): 
    all_periods_p = [row['1y percentile'], row['6m percentile'], row['3m percentile'], row['1m percentile']]
    final['HQM score'].iloc[index] = mean(all_periods_p)

#Sort values to present
final.sort_values("HQM score", ascending=False, inplace = True)
final.to_csv("momentum_advanced_HQM")

Producing data for:  AOS
                         Date       Open       High        Low      Close  \
0   2022-01-07 00:00:00-05:00  82.776511  82.923749  80.626852  81.618248   
1   2022-01-10 00:00:00-05:00  81.431755  81.726231  80.116443  81.363045   
2   2022-01-11 00:00:00-05:00  81.883278  82.472224  80.950782  82.158119   
3   2022-01-12 00:00:00-05:00  82.688158  83.169137  81.824372  82.432953   
4   2022-01-13 00:00:00-05:00  82.550736  83.385076  81.147079  81.412109   
..                        ...        ...        ...        ...        ...   
246 2022-12-30 00:00:00-05:00  57.209999  57.439999  56.689999  57.240002   
247 2023-01-03 00:00:00-05:00  57.860001  59.090000  57.450001  58.919998   
248 2023-01-04 00:00:00-05:00  59.660000  60.799999  59.419998  60.660000   
249 2023-01-05 00:00:00-05:00  60.020000  60.470001  58.930000  59.380001   
250 2023-01-06 00:00:00-05:00  60.049999  61.680000  59.849998  61.570000   

      Volume  Dividends  Stock Splits  
0    12999

<ipython-input-1-40e0ebfd3730>:30: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  y = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
<ipython-input-1-40e0ebfd3730>:31: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m6 = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
<ipython-input-1-40e0ebfd3730>:32: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m3 = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=3))
<ipython-input-1-40e0ebfd3730>:33: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime 

In [2]:
#calulate the Hurst exponent of a stock
max.set_index('Date', inplace=True)

# Get the last 6 months of data
last_6_months = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
last_6_months

<ipython-input-2-22a6b8175bf3>:5: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  last_6_months = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-07-07 00:00:00-04:00,57.665240,59.060606,57.645447,58.991333,1661300,0.0,0.0
2022-07-08 00:00:00-04:00,58.892366,58.892366,56.220392,56.378731,1516500,0.0,0.0
2022-07-11 00:00:00-04:00,56.032362,56.962608,55.824542,56.101635,1026900,0.0,0.0
2022-07-12 00:00:00-04:00,56.240188,57.150637,55.201082,55.557346,1513800,0.0,0.0
2022-07-13 00:00:00-04:00,54.547930,55.250564,53.954159,54.914093,956900,0.0,0.0
...,...,...,...,...,...,...,...
2022-12-30 00:00:00-05:00,57.209999,57.439999,56.689999,57.240002,592100,0.0,0.0
2023-01-03 00:00:00-05:00,57.860001,59.090000,57.450001,58.919998,1366200,0.0,0.0
2023-01-04 00:00:00-05:00,59.660000,60.799999,59.419998,60.660000,1054400,0.0,0.0


In [6]:
m1

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2022-12-07 00:00:00-05:00,59.250000,59.680000,59.040001,59.270000,668000,0.0,0.0
1,2022-12-08 00:00:00-05:00,59.080002,59.189999,57.950001,58.419998,1155400,0.0,0.0
2,2022-12-09 00:00:00-05:00,58.459999,59.090000,58.070000,58.299999,1074900,0.0,0.0
3,2022-12-12 00:00:00-05:00,58.299999,58.529999,57.570000,58.509998,904500,0.0,0.0
4,2022-12-13 00:00:00-05:00,60.560001,60.820000,58.919998,59.369999,1135700,0.0,0.0
5,2022-12-14 00:00:00-05:00,59.369999,60.119999,58.529999,58.880001,1209800,0.0,0.0
6,2022-12-15 00:00:00-05:00,58.040001,58.070000,56.470001,56.799999,1721800,0.0,0.0
7,2022-12-16 00:00:00-05:00,56.230000,56.549999,55.480000,56.049999,2095100,0.0,0.0
8,2022-12-19 00:00:00-05:00,55.930000,56.840000,55.750000,56.060001,754400,0.0,0.0
9,2022-12-20 00:00:00-05:00,55.830002,56.720001,55.770000,56.340000,1208700,0.0,0.0
